# Тренировка XGBoost

1. Выделить фолды
2. Преобразовать категориальные фичи
3. Сделать сетку параметров
4. Сделать 

In [19]:
import pandas as pd
import numpy as np

import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn

from sklearn.base import TransformerMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.cross_validation import KFold
from sklearn.grid_search import ParameterGrid

%matplotlib inline

In [61]:
df = pd.read_csv('../raw_data/train.csv')
df_test = pd.read_csv('../raw_data/test.csv')

In [4]:
df.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


In [64]:
df_test.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,4,A,B,A,A,A,A,A,A,B,...,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
1,6,A,B,A,B,A,A,A,A,B,...,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
2,9,A,B,A,B,B,A,B,A,B,...,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
3,12,A,A,A,A,B,A,A,A,A,...,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
4,15,B,A,A,A,A,B,A,A,A,...,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823


In [62]:
X = df.drop(['loss'], 1)
y = df.loss
X_test = df_test

In [14]:
X.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,1,A,B,A,B,A,A,A,A,B,...,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843
1,2,A,B,A,A,A,A,A,A,B,...,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496
2,5,A,B,A,A,B,A,A,A,B,...,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425
3,10,B,B,A,B,A,A,A,A,B,...,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642
4,11,A,B,A,B,A,A,A,A,B,...,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606


## Создадим 3 фолда

In [15]:
cv = KFold(X.shape[0], n_folds=3)

## Напишем класс, выделяющий фичи и преобразовывающий target, а также метрику оценки

In [73]:
class FeatureExtractor(TransformerMixin):
    def __init__(self):
        pass
        
        
    def fit(self, df_1, df_2):
        df = pd.concat([df_1, df_2], axis=0)
        self.cat_columns = [col for col in df.columns if col[:3] == 'cat']
        self.le_dict = {}
        
        for col in self.cat_columns:
            self.le_dict[col] = LabelEncoder().fit(df[col])
        
        return self
    
    def transform(self, df):
        df = df.copy()
        
        df.drop(['id'], 1, inplace=True)
        
        for col in self.cat_columns:
            df[col] = self.le_dict[col].transform(df[col])
        return df
    
    def fit_transform(self, df):
        self.fit(df)
        return self.transform(df)

In [17]:
class TargetTransformer(TransformerMixin):    
    def transform(self, y):
        return np.log(y + 200)
    
    def inverse_transform(self, y_tr):
        return np.exp(y_tr) - 200

In [74]:
fe = FeatureExtractor().fit(X, X_test)

In [75]:
X_tr = fe.transform(X)

In [76]:
X_test_tr = fe.transform(X_test)

In [28]:
X_tr.head()

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,0,1,0,1,0,0,0,0,1,0,...,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843
1,0,1,0,0,0,0,0,0,1,1,...,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496
2,0,1,0,0,1,0,0,0,1,1,...,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425
3,1,1,0,1,0,0,0,0,1,0,...,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642
4,0,1,0,1,0,0,0,0,1,1,...,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606


In [48]:
tt = TargetTransformer()

In [52]:
def mae_eval(y_true, y_pred_tr):
    y_pred = tt.inverse_transform(y_pred_tr)
    return mean_absolute_error(y_true, y_pred)

## Напишем функции для получения признаков

In [80]:
def get_train_val(X, y, train,  val):
    X_train = X.iloc[train, :]
    y_train = y.loc[train]
    y_train = tt.transform(y_train)
    X_val = X.iloc[val, :]
    y_val = y.loc[val]
    return (xgb.DMatrix(X_train, y_train.values),
            xgb.DMatrix(X_val), y_val)

In [90]:
def get_train_test(X_train, y_train, X_test):
    y_train = tt.transform(y_train)
    return (xgb.DMatrix(X_train, y_train.values),
        xgb.DMatrix(X_test))

## Функции для отправки submission

In [107]:
def get_submission(y_sub):
    df_sub = df_test[['id']].copy()
    df_sub['loss'] = y_sub
    return df_sub

## Запустим базовую модель XGB

In [51]:
params = {
    'eta': 0.1,
    'eval_metric': 'mae',
    'lambda': 1,
    'max_depth': 6,
    'objective': 'reg:linear',
    'silent': 1,
    'subsample': 1
}

In [110]:
%%time

scores = []
for train, val in cv:
    dtrain, dval, y_true = get_train_val(X_tr, y, train, val)
    watchlist = [(dtrain, 'train')]
    xgbm = xgb.train(params, dtrain, 100, evals=watchlist, 
                    early_stopping_rounds=25, verbose_eval=False)
    y_pred_tr = xgbm.predict(dval)
    score = mae_eval(y_true, y_pred_tr)
    scores.append(score)
print 'MAE = {:.2f} +- {:.2f}'.format(np.mean(scores), np.std(scores))

MAE = 1160.87 +- 1.02
CPU times: user 5min 8s, sys: 956 ms, total: 5min 9s
Wall time: 39.8 s


### Отправим  Baseline решение

In [91]:
dtrain, dtest = get_train_test(X_tr, y, X_test_tr)

In [92]:
watchlist = [(dtrain, 'train')]

In [103]:
xgbm = xgb.train(params, dtrain, 100, evals=watchlist, 
                    early_stopping_rounds=25, verbose_eval=False)
y_pred_tr = xgbm.predict(dtest)
y_pred = tt.inverse_transform(y_pred_tr)

In [108]:
submission = get_submission(y_pred)

In [109]:
submission.to_csv('../submissions/11_12_1.csv', index=False)

## Запустим обучение по сетке

scripts/xgb_gridsearch.py

Прочитаем результаты

In [1]:
with open('../run_res/xgb_gridsearch_1.tsv') as f:
    print f.read()

MAE = 3061.85	 params = {'eta': 0.01, 'silent': 1, 'eval_metric': 'mae', 'objective': 'reg:linear', 'num_boost_round': 100, 'max_depth': 4}
MAE = 1249.20	 params = {'eta': 0.01, 'silent': 1, 'eval_metric': 'mae', 'objective': 'reg:linear', 'num_boost_round': 500, 'max_depth': 4}
MAE = 1189.15	 params = {'eta': 0.01, 'silent': 1, 'eval_metric': 'mae', 'objective': 'reg:linear', 'num_boost_round': 1000, 'max_depth': 4}
MAE = 3061.01	 params = {'eta': 0.01, 'silent': 1, 'eval_metric': 'mae', 'objective': 'reg:linear', 'num_boost_round': 100, 'max_depth': 6}
MAE = 1205.88	 params = {'eta': 0.01, 'silent': 1, 'eval_metric': 'mae', 'objective': 'reg:linear', 'num_boost_round': 500, 'max_depth': 6}
MAE = 1162.13	 params = {'eta': 0.01, 'silent': 1, 'eval_metric': 'mae', 'objective': 'reg:linear', 'num_boost_round': 1000, 'max_depth': 6}
MAE = 3060.65	 params = {'eta': 0.01, 'silent': 1, 'eval_metric': 'mae', 'objective': 'reg:linear', 'num_boost_round': 100, 'max_depth': 8}
MAE = 1186.81	 par